# Final Assignment - Pick a Restaurant in Melbourne

In [1]:
import numpy as np 
import pandas as pd
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup, NavigableString, Tag
import re

print('Libraries imported.')

Libraries imported.


## Step 1 : Scraping the data from internet and parse it to the pandas dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs'

html_text =requests. get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Melbourne suburbs - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"09672f2c-6434-46f8-843b-8603de3e36a9","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Melbourne_suburbs","wgTitle":"List of Melbourne suburbs","wgCurRevisionId":1006848636,"wgRevisionId":1006848636,"wgArticleId":745628,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles with short description","Short description is different from Wikidata

In [3]:
table_contents = []

for h3 in soup.find_all('h3'):
    if h3.text.split("[")[0] == 'City of Wyndham':
        for ul in h3.find_next_sibling("ul"):
            cell ={}
            if isinstance(ul, NavigableString):
                continue
            if isinstance(ul, Tag):
                #print(ul.text)
                #print('ok')
                #print(ul.get_text())
                #print('Suburb and ZIP :', ul.text)
                #print('Suburb:', (ul.text).split("\n")[0])
                #print('area :', (ul.text).split("\n")[1:])
                cell['City'] = (h3.span.text).split("[")[0]
                cell['Suburb'] = (ul.text).split("\n")[0]
                cell['area'] = (ul.text).split("\n")[1:]
                table_contents.append(cell)
        
        #print('h3',h3)
        break
    else:
        #print((h3.span.text).split("[")[0])
        
    #print(h3)
        for ul in h3.find_next_sibling("ul"):
            cell ={}
            if isinstance(ul, NavigableString):
                continue
            if isinstance(ul, Tag):
                #print(ul.text)
                #print('ok')
                #print(ul.get_text())
                #print('Suburb and ZIP :', ul.text)
                #print('Suburb:', (ul.text).split("\n")[0])
                #print('area :', (ul.text).split("\n")[1:])
                cell['City'] = (h3.span.text).split("[")[0]
                cell['Suburb'] = (ul.text).split("\n")[0]
                cell['area'] = (ul.text).split("\n")[1:]
                print(cell)
                print('ok')
                table_contents.append(cell)
            
#table_contents


{'City': 'City of Melbourne', 'Suburb': 'Carlton 3053', 'area': ['Carlton South']}
ok
{'City': 'City of Melbourne', 'Suburb': 'Carlton North 3054 (Shared with City of Yarra)', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'Docklands 3008', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'East Melbourne 3002', 'area': ['Jolimont  (the name of the railway station)']}
ok
{'City': 'City of Melbourne', 'Suburb': 'Flemington 3031 (Shared with City of Moonee Valley)', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'Kensington 3031', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'Melbourne 3000 (Central business district)', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'Melbourne 3004 (St Kilda Road area, shared with City of Port Phillip)', 'area': []}
ok
{'City': 'City of Melbourne', 'Suburb': 'North Melbourne 3051 (Shared with City of Moonee Valley)', 'area': ['Hotham Hill', 'Macaulay (the name of the railway station)']}
ok
{'City': 'City of Melbourn

In [4]:
#table_contents
df=pd.DataFrame(table_contents)
df.head()    # It is missing rurual localities.
#print (type(df['area']))

,City,Suburb,area
0,City of Melbourne,Carlton 3053,[Carlton South]
1,City of Melbourne,Carlton North 3054 (Shared with City of Yarra),[]
2,City of Melbourne,Docklands 3008,[]
3,City of Melbourne,East Melbourne 3002,[Jolimont (the name of the railway station)]
4,City of Melbourne,Flemington 3031 (Shared with City of Moonee Va...,[]


In [5]:
df=pd.DataFrame(table_contents)
df.head(20)    # It is missing rurual localities.

df['Suburb']=df['Suburb'].str.replace(r'\s\(.*\)','')
df['PostalCode']=df['Suburb'].str[-4:]
df['Suburb']=df['Suburb'].str[:-5]

df['area'] = df['area'].astype(str).str[1:-1]  # remove the square brackets

df['area']=df['area'].str.replace(r'\s\(.*\)','')   # remove the comments in '()'

df.head(20)

<ipython-input-5-6903895418c4>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Suburb']=df['Suburb'].str.replace(r'\s\(.*\)','')
<ipython-input-5-6903895418c4>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['area']=df['area'].str.replace(r'\s\(.*\)','')   # remove the comments in '()'


,City,Suburb,area,PostalCode
0,City of Melbourne,Carlton,'Carlton South',3053
1,City of Melbourne,Carlton North,,3054
2,City of Melbourne,Docklands,,3008
3,City of Melbourne,East Melbourne,'Jolimont ',3002
4,City of Melbourne,Flemington,,3031
5,City of Melbourne,Kensington,,3031
6,City of Melbourne,Melbourne,,3000
7,City of Melbourne,Melbourne,,3004
8,City of Melbourne,North Melbourne,"'Hotham Hill', 'Macaulay'",3051
9,City of Melbourne,Parkville,'Royal Park',3052


In [6]:
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['area'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'City': np.repeat(df['City'], lens),
                    'Suburb': np.repeat(df['Suburb'], lens),
                    'PostalCode': np.repeat(df['PostalCode'], lens),
                    'area': chainer(df['area'])})

res=res.reset_index()
res=res.drop(columns='index')

res['area']=res['area'].replace({'\'': ''}, regex=True)   # remove single quote sign
res['area']=res['area'].replace({'\"': ''}, regex=True)   # remove double quote sign

#res['area'].fillna(res.Suburb, inplace=True)
#res.loc[res['area'].isna(),'area'] = res['Suburb']
res['area']=np.where(res['area']=='',res['Suburb'],res['area'])

res.head(20)

,City,Suburb,PostalCode,area
0,City of Melbourne,Carlton,3053,Carlton South
1,City of Melbourne,Carlton North,3054,Carlton North
2,City of Melbourne,Docklands,3008,Docklands
3,City of Melbourne,East Melbourne,3002,Jolimont
4,City of Melbourne,Flemington,3031,Flemington
5,City of Melbourne,Kensington,3031,Kensington
6,City of Melbourne,Melbourne,3000,Melbourne
7,City of Melbourne,Melbourne,3004,Melbourne
8,City of Melbourne,North Melbourne,3051,Hotham Hill
9,City of Melbourne,North Melbourne,3051,Macaulay


## Step 2 : Use the folium and geocoder to generate the local map


In [7]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

latitude=[]
longitude=[]
for code in res['Suburb']:
    g = geocoder.arcgis('{}, Melbourne, VIC'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Melbourne, VIC'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

Carlton [-37.80100999999996, 144.96951]
Carlton North [-37.78917999999993, 144.97188000000006]
Docklands [-37.81925999999993, 144.94555000000003]
East Melbourne [-37.81131999999997, 144.9779900000001]
Flemington [-37.78813999999994, 144.92965000000004]
Kensington [-37.794209999999964, 144.92754000000002]
Melbourne [-37.81738999999993, 144.96751000000006]
Melbourne [-37.81738999999993, 144.96751000000006]
North Melbourne [-37.800539999999955, 144.94609000000003]
North Melbourne [-37.800539999999955, 144.94609000000003]
Parkville [-37.79273999999998, 144.95825000000002]
Port Melbourne [-37.833009999999945, 144.92434000000003]
Port Melbourne [-37.833009999999945, 144.92434000000003]
Southbank [-37.82523999999995, 144.96093000000008]
South Wharf [-37.825909999999965, 144.9515100000001]
South Yarra [-37.83911999999998, 144.9927100000001]
West Melbourne [-37.80670999999995, 144.9503800000001]
Albert Park [-37.84403999999995, 144.95126000000005]
Balaclava [-37.86902999999995, 144.995650000000

Epping [-37.646199999999965, 145.02618000000007]
Epping [-37.646199999999965, 145.02618000000007]
Lalor [-37.67130999999995, 145.01555000000008]
Mernda [-37.58706999999998, 145.1031700000001]
Mill Park [-37.657159999999976, 145.0751600000001]
South Morang [-37.63704999999993, 145.09567000000004]
Thomastown [-37.67784999999998, 145.01431000000002]
Ashburton [-37.863099999999974, 145.07716000000005]
Balwyn [-37.81149999999997, 145.08159000000012]
Balwyn [-37.81149999999997, 145.08159000000012]
Balwyn North [-37.793889999999976, 145.0779000000001]
Balwyn North [-37.793889999999976, 145.0779000000001]
Balwyn North [-37.793889999999976, 145.0779000000001]
Balwyn North [-37.793889999999976, 145.0779000000001]
Balwyn North [-37.793889999999976, 145.0779000000001]
Camberwell [-37.83629999999994, 145.0625100000001]
Camberwell [-37.83629999999994, 145.0625100000001]
Camberwell [-37.83629999999994, 145.0625100000001]
Camberwell [-37.83629999999994, 145.0625100000001]
Canterbury [-37.8252199999999

Dandenong North [-37.95420999999993, 145.21402000000012]
Dandenong South [-38.01232999999996, 145.20595000000003]
Keysborough [-37.992339999999956, 145.1754400000001]
Noble Park [-37.96696999999995, 145.1899800000001]
Noble Park [-37.96696999999995, 145.1899800000001]
Noble Park [-37.96696999999995, 145.1899800000001]
Noble Park North [-37.94745999999998, 145.19180000000006]
Springvale [-37.94751999999994, 145.1531500000001]
Springvale South [-37.978549999999984, 145.1472500000001]
Carrum Downs [-38.10092999999995, 145.1829100000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston [-38.14117999999996, 145.1217600000001]
Frankston 

Sydenham [-37.701809999999966, 144.76289000000008]
Taylors Lakes [-37.69667999999996, 144.78138]
Taylors Lakes [-37.69667999999996, 144.78138]
Tullamarine [-37.71355999999997, 144.8687000000001]
Altona [-37.863819999999976, 144.82482000000005]
Altona Meadows [-37.87176999999997, 144.77760000000012]
Altona North [-37.830469999999934, 144.84134000000006]
Altona North [-37.830469999999934, 144.84134000000006]
Brooklyn [-37.82017999999994, 144.84927000000005]
Laverton [-37.86230999999998, 144.76976000000002]
Newport [-37.84259999999995, 144.88195000000007]
Spotswood [-37.82965999999993, 144.88517000000002]
Seabrook [-37.878289999999936, 144.75764000000004]
Seaholme [-37.86777999999998, 144.84377000000006]
South Kingsville [-37.83365999999995, 144.87146000000007]
Williamstown [-37.86079999999998, 144.90235000000007]
Williamstown North [-37.851639999999975, 144.86787000000004]
Braybrook [-37.780209999999954, 144.85649000000012]
Footscray [-37.791339999999934, 144.8899600000001]
Kingsville [-

In [8]:
res['Latitude'] = latitude
res['Longitude'] = longitude
res.head(20)

,City,Suburb,PostalCode,area,Latitude,Longitude
0,City of Melbourne,Carlton,3053,Carlton South,-37.80101,144.96951
1,City of Melbourne,Carlton North,3054,Carlton North,-37.78918,144.97188
2,City of Melbourne,Docklands,3008,Docklands,-37.81926,144.94555
3,City of Melbourne,East Melbourne,3002,Jolimont,-37.81132,144.97799
4,City of Melbourne,Flemington,3031,Flemington,-37.78814,144.92965
5,City of Melbourne,Kensington,3031,Kensington,-37.79421,144.92754
6,City of Melbourne,Melbourne,3000,Melbourne,-37.81739,144.96751
7,City of Melbourne,Melbourne,3004,Melbourne,-37.81739,144.96751
8,City of Melbourne,North Melbourne,3051,Hotham Hill,-37.80054,144.94609
9,City of Melbourne,North Melbourne,3051,Macaulay,-37.80054,144.94609


In [12]:
address = 'Melbourne, VIC'

geolocator = Nominatim(user_agent="Suburb_Melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [27]:
# create map of Manhattan using latitude and longitude values
map_Melbourne = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, suburb, neighborhood in zip(res['Latitude'], res['Longitude'], res['Suburb'], res['area']):
    label = '{}, {}'.format(neighborhood, suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Melbourne)  
    
map_Melbourne

In [75]:
# Local_data = res[res['City'] == 'City of Melbourne'].reset_index(drop=True)
# Local_data
Local_data = res
Local_data

,City,Suburb,PostalCode,area,Latitude,Longitude
0,City of Melbourne,Carlton,3053,Carlton South,-37.80101,144.96951
1,City of Melbourne,Carlton North,3054,Carlton North,-37.78918,144.97188
2,City of Melbourne,Docklands,3008,Docklands,-37.81926,144.94555
3,City of Melbourne,East Melbourne,3002,Jolimont,-37.81132,144.97799
4,City of Melbourne,Flemington,3031,Flemington,-37.78814,144.92965
...,...,...,...,...,...,...
557,City of Wyndham,Werribee,3030,Brophys Crossing,-37.90319,144.66054
558,City of Wyndham,Werribee,3030,Mossfield,-37.90319,144.66054
559,City of Wyndham,Werribee South,3030,Werribee South,-37.91207,144.68203
560,City of Wyndham,Williams Landing,3027,Williams RAAF,-37.85740,144.73788


In [76]:
address = 'City of Melbourne, Melbourne'

geolocator = Nominatim(user_agent="City_of_Melbourne")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of City of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of City of Melbourne are -37.8142176, 144.9631608.


In [77]:
# create map of Manhattan using latitude and longitude values
map_City_of_Melbourne = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, suburb, neighborhood in zip(Local_data['Latitude'], Local_data['Longitude'], Local_data['Suburb'], Local_data['area']):
    label = '{}, {}'.format(neighborhood, suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_City_of_Melbourne)  
    
map_City_of_Melbourne

# Step 3 Use Foursquare data to study the venues of the local suburbs

In [78]:
CLIENT_ID = 'MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE' # your Foursquare ID
CLIENT_SECRET = '2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE
CLIENT_SECRET:2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y


In [79]:
Local_data.loc[0, 'Suburb']

'Carlton'

In [80]:
neighborhood_latitude = Local_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Local_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Local_data.loc[0, 'Suburb'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Carlton are -37.80100999999996, 144.96951.


In [81]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE&client_secret=2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y&v=20180605&ll=-37.80100999999996,144.96951&radius=500&limit=100'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6077e137017a3270f519c401'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Carlton',
  'headerFullLocation': 'Carlton, Melbourne',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': -37.79650999549996,
    'lng': 144.97519453508994},
   'sw': {'lat': -37.805510004499965, 'lng': 144.9638254649101}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0b8439f964a520053223e3',
       'name': 'D.O.C. Pizza & Mozzarella Bar',
       'location': {'address': '295 Drummond St',
        'crossStreet': 'at Faraday St',
        'lat': -37.798954,
        'lng': 144.96849,
        'labeledLatLngs': [{'lab

In [83]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [84]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-84-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,D.O.C. Pizza & Mozzarella Bar,Pizza Place,-37.798954,144.968490
1,Carlton Wine Room,Wine Bar,-37.798584,144.968610
2,Baker D. Chirico,Bakery,-37.798788,144.968499
3,Gewürzhaus,Gourmet Shop,-37.799050,144.967480
4,Assembly Coffee & Tea,Coffee Shop,-37.802750,144.967290


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
Local_venues = getNearbyVenues(names=Local_data['Suburb'],
                                   latitudes=Local_data['Latitude'],
                                   longitudes=Local_data['Longitude']
                                  )

Carlton
Carlton North
Docklands
East Melbourne
Flemington
Kensington
Melbourne
Melbourne
North Melbourne
North Melbourne
Parkville
Port Melbourne
Port Melbourne
Southbank
South Wharf
South Yarra
West Melbourne
Albert Park
Balaclava
Elwood
Melbourne
Middle Park
Port Melbourne
Port Melbourne
Port Melbourne
Port Melbourne
Port Melbourne
Port Melbourne
Ripponlea
St Kilda
St Kilda East
St Kilda West
Southbank
South Melbourne
South Melbourne
South Melbourne
South Wharf
Abbotsford
Alphington
Burnley
Carlton North
Clifton Hill
Collingwood
Cremorne
Fairfield
Fitzroy
Fitzroy
Fitzroy North
Fitzroy North
Princes Hill
Richmond
Richmond
Bellfield
Briar Hill
Bundoora
Bundoora
Bundoora
Bundoora
Eaglemont
Eltham
Eltham North
Greensborough
Greensborough
Greensborough
Heidelberg
Heidelberg Heights
Heidelberg West
Ivanhoe
Ivanhoe
Ivanhoe
Ivanhoe East
Lower Plenty
Macleod
Macleod
Montmorency
Rosanna
St Helena
Viewbank
Watsonia
Watsonia North
Yallambie
Alphington
Bundoora
Coburg
Coburg North
Fairfield
Kings

In [88]:
print(Local_venues.shape)
Local_venues.head()

(8169, 7)


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Carlton,-37.80101,144.96951,D.O.C. Pizza & Mozzarella Bar,-37.798954,144.968490,Pizza Place
1,Carlton,-37.80101,144.96951,Carlton Wine Room,-37.798584,144.968610,Wine Bar
2,Carlton,-37.80101,144.96951,Baker D. Chirico,-37.798788,144.968499,Bakery
3,Carlton,-37.80101,144.96951,Gewürzhaus,-37.799050,144.967480,Gourmet Shop
4,Carlton,-37.80101,144.96951,Assembly Coffee & Tea,-37.802750,144.967290,Coffee Shop


In [89]:
print('There are {} uniques categories.'.format(len(Local_venues['Venue Category'].unique())))

There are 339 uniques categories.


In [134]:
# one hot encoding
Local_onehot = pd.get_dummies(Local_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Local_onehot['Suburb'] = Local_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [Local_onehot.columns[-1]] + list(Local_onehot.columns[:-1])
Local_onehot = Local_onehot[fixed_columns]

Local_onehot.head()

,Suburb,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant
0,Carlton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Carlton,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Carlton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Carlton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
Local_onehot.shape

(8169, 340)

In [136]:
Local_grouped = Local_onehot.groupby('Suburb').mean().reset_index()
Local_grouped

,Suburb,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant
0,,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.01,0.01,0.00,0.0,0.0,0.0,0.00,0.0
1,Abbotsford,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0
2,Aberfeldie,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0
3,Ai,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.01,0.01,0.00,0.0,0.0,0.0,0.00,0.0
4,Airport West,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,Windsor,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.02,0.0
357,Wonga Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0
358,Wyndham Vale,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0
359,Yallambie,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0


In [137]:
num_top_venues = 5

for hood in Local_grouped['Suburb']:
    print("----"+hood+"----")
    temp = Local_grouped[Local_grouped['Suburb'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

--------
              venue  freq
0               Bar  0.10
1              Café  0.05
2  Asian Restaurant  0.04
3      Cocktail Bar  0.04
4       Coffee Shop  0.04


----Abbotsford----
              venue  freq
0              Café  0.19
1               Pub  0.14
2       Pizza Place  0.10
3  Greek Restaurant  0.05
4       Coffee Shop  0.05


----Aberfeldie----
                  venue  freq
0          Tennis Court   1.0
1     Accessories Store   0.0
2  Other Great Outdoors   0.0
3           Pastry Shop   0.0
4                  Park   0.0


----Ai----
              venue  freq
0               Bar  0.10
1              Café  0.05
2  Asian Restaurant  0.04
3      Cocktail Bar  0.04
4       Coffee Shop  0.04


----Airport West----
                  venue  freq
0      Department Store  0.08
1           Coffee Shop  0.08
2           Supermarket  0.08
3                  Café  0.08
4  Fast Food Restaurant  0.08


----Albanvale----
                    venue  freq
0  Furniture / Home Store   1.0
1

                    venue  freq
0      Italian Restaurant  0.14
1             Pizza Place  0.14
2  Furniture / Home Store  0.14
3    Gym / Fitness Center  0.14
4              Food Truck  0.14


----Briar Hill----
                      venue  freq
0        Seafood Restaurant  0.33
1  Mediterranean Restaurant  0.33
2        Photography Studio  0.33
3         Accessories Store  0.00
4          Pedestrian Plaza  0.00


----Brighton East----
               venue  freq
0           Bus Stop   1.0
1  Accessories Store   0.0
2          Pawn Shop   0.0
3        Pastry Shop   0.0
4               Park   0.0


----Broadmeadows----
               venue  freq
0             Bakery  0.50
1      Shopping Mall  0.25
2      Grocery Store  0.25
3  Accessories Store  0.00
4        Music Venue  0.00


----Brookfield----
                venue  freq
0                Café  0.08
1                 Bar  0.06
2  Italian Restaurant  0.05
3  Mexican Restaurant  0.04
4  Chinese Restaurant  0.04


----Brooklyn----
    

4               Liquor Store  0.05


----Coburg North----
                    venue  freq
0                 RV Park   0.2
1             Dry Cleaner   0.2
2          Scenic Lookout   0.2
3  Thrift / Vintage Store   0.2
4              Print Shop   0.2


----Collingwood----
          venue  freq
0          Café  0.13
1  Cocktail Bar  0.07
2           Pub  0.06
3   Coffee Shop  0.06
4        Bakery  0.06


----Coolaroo----
                    venue  freq
0                     Pub   0.2
1  Furniture / Home Store   0.2
2            Dessert Shop   0.2
3              Restaurant   0.2
4            Home Service   0.2


----Craigieburn----
                venue  freq
0  Italian Restaurant   0.2
1       Train Station   0.2
2    Kebab Restaurant   0.2
3                 Gym   0.2
4         Chaat Place   0.2


----Cranbourne----
                  venue  freq
0           Gas Station  0.50
1  Fast Food Restaurant  0.25
2         Grocery Store  0.25
3     Accessories Store  0.00
4  Other Great Outdoors 

              venue  freq
0              Café  0.17
1    Clothing Store  0.17
2      Liquor Store  0.17
3  Asian Restaurant  0.17
4       Pizza Place  0.17


----Hurstbridge----
               venue  freq
0               Café  0.43
1      Train Station  0.29
2      Grocery Store  0.14
3        Pizza Place  0.14
4  Accessories Store  0.00


----Ivanhoe----
                   venue  freq
0    Japanese Restaurant  0.13
1       Sushi Restaurant  0.07
2                    Bar  0.07
3                    Gym  0.07
4  Performing Arts Venue  0.07


----Ivanhoe East----
                venue  freq
0  Italian Restaurant  0.17
1                Park  0.17
2              Bakery  0.17
3         Supermarket  0.17
4                Café  0.17


----Jacana----
                     venue  freq
0               Playground  0.33
1              Pizza Place  0.33
2  Health & Beauty Service  0.33
3            National Park  0.00
4          Nature Preserve  0.00


----Junction Village----
               venue  f

4           Pastry Shop   0.0


----Narre Warren South----
           venue  freq
0    Supermarket  0.29
1  Shopping Mall  0.14
2    Pizza Place  0.14
3         Bakery  0.14
4   Burger Joint  0.14


----Newport----
                  venue  freq
0                  Café  0.21
1           Pizza Place  0.21
2        Ice Cream Shop  0.07
3       Thai Restaurant  0.07
4  Fast Food Restaurant  0.07


----Niddrie----
                venue  freq
0                Café  0.25
1  Italian Restaurant  0.17
2   Convenience Store  0.08
3     Thai Restaurant  0.08
4    Asian Restaurant  0.08


----Noble Park----
                venue  freq
0         Pizza Place   0.4
1        Liquor Store   0.2
2  Chinese Restaurant   0.2
3                Park   0.2
4   Accessories Store   0.0


----Noble Park North----
                  venue  freq
0              Boutique   0.2
1   Fried Chicken Joint   0.2
2      Kebab Restaurant   0.2
3  Fast Food Restaurant   0.2
4           Pizza Place   0.2


----North Melbourne--

                        venue  freq
0  Construction & Landscaping  0.33
1                 Supermarket  0.33
2                 Golf Course  0.33
3           Accessories Store  0.00
4        Other Great Outdoors  0.00


----Sandhurst----
               venue  freq
0        Golf Course  0.25
1               Park  0.25
2      Women's Store  0.25
3               Café  0.25
4  Accessories Store  0.00


----Sassafras----
                    venue  freq
0                    Café  0.50
1                Tea Room  0.25
2        Toy / Game Store  0.12
3             Coffee Shop  0.12
4  Peking Duck Restaurant  0.00


----Scoresby----
                    venue  freq
0              Hookah Bar   1.0
1       Accessories Store   0.0
2  Peking Duck Restaurant   0.0
3               Pawn Shop   0.0
4             Pastry Shop   0.0


----Seabrook----
               venue  freq
0        Pizza Place  0.50
1  Indian Restaurant  0.25
2  Fish & Chips Shop  0.25
3  Accessories Store  0.00
4    Other Nightlife  0.0

               venue  freq
0         Playground  0.33
1               Park  0.33
2           Bus Stop  0.33
3  Accessories Store  0.00
4        Pastry Shop  0.00


----Yallambie----
                    venue  freq
0               Locksmith  0.33
1                    Park  0.33
2                  Bakery  0.33
3       Accessories Store  0.00
4  Peking Duck Restaurant  0.00


----Yarraville----
                venue  freq
0                Café  0.29
1         Pizza Place  0.09
2       Grocery Store  0.06
3  Frozen Yogurt Shop  0.06
4                Park  0.03




In [138]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [167]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Suburb'] = Local_grouped['Suburb']

for ind in np.arange(Local_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Local_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,,Bar,Café,Asian Restaurant,Cocktail Bar,Coffee Shop,Theater,Boutique,Hotel,Italian Restaurant,Burger Joint
1,Abbotsford,Café,Pub,Pizza Place,Greek Restaurant,Coffee Shop,Grocery Store,Rock Climbing Spot,Bus Stop,Convenience Store,Gay Bar
2,Aberfeldie,Tennis Court,Accessories Store,Other Great Outdoors,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
3,Ai,Bar,Café,Asian Restaurant,Cocktail Bar,Coffee Shop,Theater,Boutique,Hotel,Italian Restaurant,Burger Joint
4,Airport West,Department Store,Coffee Shop,Supermarket,Café,Fast Food Restaurant,Electronics Store,Clothing Store,Sandwich Place,Tram Station,Shopping Mall


In [196]:
# set number of clusters
kclusters = 5

Local_grouped_clustering = Local_grouped.drop('Suburb', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Local_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 3, 4, 0, 0, 0, 3], dtype=int32)

In [169]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Local_merged = Local_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Local_merged = Local_merged.join(neighborhoods_venues_sorted.set_index('Suburb'), on='Suburb')

Local_merged.head() # check the last columns!

,City,Suburb,PostalCode,area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of Melbourne,Carlton,3053,Carlton South,-37.80101,144.96951,3.0,Italian Restaurant,Café,Coffee Shop,Ice Cream Shop,French Restaurant,Gourmet Shop,Vegetarian / Vegan Restaurant,Thai Restaurant,Wine Bar,Cheese Shop
1,City of Melbourne,Carlton North,3054,Carlton North,-37.78918,144.97188,3.0,Pub,Café,Grocery Store,Italian Restaurant,Chiropractor,BBQ Joint,Bakery,Cocktail Bar,Thai Restaurant,Shoe Store
2,City of Melbourne,Docklands,3008,Docklands,-37.81926,144.94555,0.0,Café,Coffee Shop,Italian Restaurant,Vietnamese Restaurant,Bar,Salad Place,Stadium,Thai Restaurant,Middle Eastern Restaurant,Tapas Restaurant
3,City of Melbourne,East Melbourne,3002,Jolimont,-37.81132,144.97799,0.0,Café,Wine Bar,Cocktail Bar,Park,Coffee Shop,Hotel,Garden,Mexican Restaurant,Hostel,Theater
4,City of Melbourne,Flemington,3031,Flemington,-37.78814,144.92965,3.0,Café,Bakery,Pub,Malay Restaurant,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Dessert Shop,Tram Station,Coffee Shop


In [170]:
print(Local_merged['Cluster Labels'])

#Local_merged = Local_merged[Local_merged['Cluster Labels'].notna()]

Local_merged.dropna(subset=['Cluster Labels'],inplace=True)

Local_merged= Local_merged.reset_index()
Local_merged=Local_merged.drop(columns='index')
Local_merged
#Local_merged.iloc[560]
#Local_merged.loc[Local_merged['Cluster Labels'] == 0

0      3.0
1      3.0
2      0.0
3      0.0
4      3.0
      ... 
557    3.0
558    3.0
559    3.0
560    NaN
561    1.0
Name: Cluster Labels, Length: 562, dtype: float64


,City,Suburb,PostalCode,area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,City of Melbourne,Carlton,3053,Carlton South,-37.80101,144.96951,3.0,Italian Restaurant,Café,Coffee Shop,Ice Cream Shop,French Restaurant,Gourmet Shop,Vegetarian / Vegan Restaurant,Thai Restaurant,Wine Bar,Cheese Shop
1,City of Melbourne,Carlton North,3054,Carlton North,-37.78918,144.97188,3.0,Pub,Café,Grocery Store,Italian Restaurant,Chiropractor,BBQ Joint,Bakery,Cocktail Bar,Thai Restaurant,Shoe Store
2,City of Melbourne,Docklands,3008,Docklands,-37.81926,144.94555,0.0,Café,Coffee Shop,Italian Restaurant,Vietnamese Restaurant,Bar,Salad Place,Stadium,Thai Restaurant,Middle Eastern Restaurant,Tapas Restaurant
3,City of Melbourne,East Melbourne,3002,Jolimont,-37.81132,144.97799,0.0,Café,Wine Bar,Cocktail Bar,Park,Coffee Shop,Hotel,Garden,Mexican Restaurant,Hostel,Theater
4,City of Melbourne,Flemington,3031,Flemington,-37.78814,144.92965,3.0,Café,Bakery,Pub,Malay Restaurant,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Dessert Shop,Tram Station,Coffee Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,City of Wyndham,Truganina,3029,Truganina,-37.85209,144.72399,3.0,Bus Stop,Fast Food Restaurant,Accessories Store,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
535,City of Wyndham,Werribee,3030,Brophys Crossing,-37.90319,144.66054,3.0,Café,Asian Restaurant,Pizza Place,Fast Food Restaurant,Indian Restaurant,Middle Eastern Restaurant,Bar,Gym / Fitness Center,Performing Arts Venue,Paper / Office Supplies Store
536,City of Wyndham,Werribee,3030,Mossfield,-37.90319,144.66054,3.0,Café,Asian Restaurant,Pizza Place,Fast Food Restaurant,Indian Restaurant,Middle Eastern Restaurant,Bar,Gym / Fitness Center,Performing Arts Venue,Paper / Office Supplies Store
537,City of Wyndham,Werribee South,3030,Werribee South,-37.91207,144.68203,3.0,Gym / Fitness Center,Restaurant,Accessories Store,Other Great Outdoors,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture


In [190]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

print(type(cluster)) # there us NaN in cluster

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Local_merged['Latitude'], Local_merged['Longitude'], Local_merged['Suburb'], Local_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.5
    ).add_to(map_clusters)
       
map_clusters

<class 'float'>


In [191]:
map_clusters.save('plot_data.html')

# Step 4 Classifications of the local venue types

# Class 0: Restaurants diversed (high density)

In [176]:
Local_merged.loc[Local_merged['Cluster Labels'] == 0, Local_merged.columns[[1] + list(range(5, Local_merged.shape[1]))]]


,Suburb,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Docklands,144.94555,0.0,Café,Coffee Shop,Italian Restaurant,Vietnamese Restaurant,Bar,Salad Place,Stadium,Thai Restaurant,Middle Eastern Restaurant,Tapas Restaurant
3,East Melbourne,144.97799,0.0,Café,Wine Bar,Cocktail Bar,Park,Coffee Shop,Hotel,Garden,Mexican Restaurant,Hostel,Theater
5,Kensington,144.92754,0.0,Café,Park,Japanese Restaurant,Pub,Fish & Chips Shop,Burger Joint,Bar,Supermarket,Ice Cream Shop,Gym
8,North Melbourne,144.94609,0.0,Café,Pub,Garden,Pool,Sports Club,Bar,Malay Restaurant,Electronics Store,Middle Eastern Restaurant,Convenience Store
9,North Melbourne,144.94609,0.0,Café,Pub,Garden,Pool,Sports Club,Bar,Malay Restaurant,Electronics Store,Middle Eastern Restaurant,Convenience Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,Keilor East,144.86360,0.0,Spa,Café,Accessories Store,Other Great Outdoors,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
523,Moonee Ponds,144.92265,0.0,Café,Italian Restaurant,Supermarket,Tram Station,Noodle House,Grocery Store,Gym,Shopping Mall,Food & Drink Shop,Park
524,Niddrie,144.88795,0.0,Café,Italian Restaurant,Convenience Store,Thai Restaurant,Asian Restaurant,Grocery Store,Turkish Restaurant,Baby Store,Portuguese Restaurant,Office
527,Travancore,144.93285,0.0,Escape Room,Farmers Market,Fast Food Restaurant,Café,Accessories Store,Peking Duck Restaurant,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store


# Class 1: Play ground and local food

In [177]:
Local_merged.loc[Local_merged['Cluster Labels'] == 1, Local_merged.columns[[1] + list(range(5, Local_merged.shape[1]))]]


,Suburb,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Lower Plenty,145.123390,1.0,Playground,Church,Home Service,Performing Arts Venue,Newsstand,Outdoor Supply Store,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store
108,Jacana,144.913510,1.0,Playground,Pizza Place,Health & Beauty Service,National Park,Nature Preserve,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall
237,Warranwood,145.248290,1.0,Playground,Fish & Chips Shop,Accessories Store,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
295,Junction Village,145.294830,1.0,Playground,Garden Center,Convenience Store,Accessories Store,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
351,Carrum,145.133910,1.0,Playground,Sports Club,Pub,Organic Grocery,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
360,Edithvale,145.114720,1.0,Playground,Bus Stop,Accessories Store,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
370,Patterson Lakes,145.145210,1.0,Home Service,Accessories Store,Peking Duck Restaurant,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
401,Blairgowrie,144.772244,1.0,Home Service,Beach,Accessories Store,Other Nightlife,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store
441,Delahey,144.770630,1.0,Playground,Accessories Store,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife,Other Great Outdoors
492,Exford,145.187910,1.0,Playground,Fast Food Restaurant,Accessories Store,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife


# Class 2: Park/ Convinient store/ Business Services 

In [178]:
Local_merged.loc[Local_merged['Cluster Labels'] == 2, Local_merged.columns[[1] + list(range(5, Local_merged.shape[1]))]]


,Suburb,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,St Helena,145.13149,2.0,Convenience Store,Park,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
76,Viewbank,145.09629,2.0,Park,Bakery,Accessories Store,Other Great Outdoors,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
79,Yallambie,145.09538,2.0,Locksmith,Park,Bakery,Accessories Store,Peking Duck Restaurant,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store
107,Greenvale,144.88962,2.0,Other Great Outdoors,Park,Accessories Store,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
143,Plenty,145.12399,2.0,Park,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
205,Doncaster,145.13224,2.0,Park,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
206,Doncaster,145.13224,2.0,Park,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
214,Park Orchards,145.21786,2.0,Tour Provider,Business Service,Park,Burger Joint,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
219,Wonga Park,145.27157,2.0,Park,Tennis Court,Accessories Store,Other Great Outdoors,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture,Other Nightlife
231,Park Orchards,145.21786,2.0,Tour Provider,Business Service,Park,Burger Joint,Pawn Shop,Pastry Shop,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture


# Class 3: Cafe/ Resteraunts/ Pubs (Medium density)

In [179]:
Local_merged.loc[Local_merged['Cluster Labels'] == 3, Local_merged.columns[[1] + list(range(5, Local_merged.shape[1]))]]



,Suburb,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Carlton,144.96951,3.0,Italian Restaurant,Café,Coffee Shop,Ice Cream Shop,French Restaurant,Gourmet Shop,Vegetarian / Vegan Restaurant,Thai Restaurant,Wine Bar,Cheese Shop
1,Carlton North,144.97188,3.0,Pub,Café,Grocery Store,Italian Restaurant,Chiropractor,BBQ Joint,Bakery,Cocktail Bar,Thai Restaurant,Shoe Store
4,Flemington,144.92965,3.0,Café,Bakery,Pub,Malay Restaurant,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Dessert Shop,Tram Station,Coffee Shop
6,Melbourne,144.96751,3.0,Bar,Café,Asian Restaurant,Cocktail Bar,Coffee Shop,Theater,Boutique,Hotel,Italian Restaurant,Burger Joint
7,Melbourne,144.96751,3.0,Bar,Café,Asian Restaurant,Cocktail Bar,Coffee Shop,Theater,Boutique,Hotel,Italian Restaurant,Burger Joint
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,Tarneit,144.67290,3.0,Sports Club,Garden,Pharmacy,Discount Store,Home Service,Asian Restaurant,Pizza Place,Accessories Store,Other Nightlife,Park
534,Truganina,144.72399,3.0,Bus Stop,Fast Food Restaurant,Accessories Store,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
535,Werribee,144.66054,3.0,Café,Asian Restaurant,Pizza Place,Fast Food Restaurant,Indian Restaurant,Middle Eastern Restaurant,Bar,Gym / Fitness Center,Performing Arts Venue,Paper / Office Supplies Store
536,Werribee,144.66054,3.0,Café,Asian Restaurant,Pizza Place,Fast Food Restaurant,Indian Restaurant,Middle Eastern Restaurant,Bar,Gym / Fitness Center,Performing Arts Venue,Paper / Office Supplies Store


# Class 4: furniture/Accessories/store focused.

In [180]:
Local_merged.loc[Local_merged['Cluster Labels'] == 4, Local_merged.columns[[1] + list(range(5, Local_merged.shape[1]))]]


,Suburb,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
435,Albanvale,144.77025,4.0,Furniture / Home Store,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
531,Manor Lakes,144.60274,4.0,Furniture / Home Store,Accessories Store,Other Great Outdoors,Pawn Shop,Pastry Shop,Park,Paper / Office Supplies Store,Outlet Mall,Outdoor Supply Store,Outdoor Sculpture
